## Importing the relevant packages

In [1]:
%pip install arch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats.distributions import chi2 
from arch import arch_model
from math import sqrt
import seaborn as sns
sns.set()

## Importing the Data and Pre-processing 

In [3]:
raw_csv_data = pd.read_csv("Index2023.csv") 
df_comp=raw_csv_data.copy()
df_comp.date = pd.to_datetime(df_comp.date, dayfirst = False)
df_comp.set_index("date", inplace=True)
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

C:\Users\heber\AppData\Local\Temp\ipykernel_15196\1907149261.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_comp=df_comp.fillna(method='ffill')


In [4]:
df_comp['market_value']=df_comp.ftse

In [5]:
del df_comp['spx']
del df_comp['dax']
del df_comp['ftse']
del df_comp['nikkei']
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [6]:
import warnings
warnings.filterwarnings("ignore")

## The LLR Test

In [7]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.fit(start_ar_lags = 11).llf
    L2 = mod_2.fit(start_ar_lags = 11).llf
    LR = (2*(L2-L1))    
    p = chi2.sf(LR, DF).round(4)
    return p

## Creating Returns

In [8]:
df['returns'] = df.market_value.pct_change(1)*100

## The Simple GARCH Model

In [10]:
#p=1: El orden del modelo GARCH para el componente autoregresivo (ARCH)
#q=1: El orden del modelo GARCH para el componente de media móvil (GARCH)
model_garch_1_1 = arch_model(df.returns[1:], mean = "Constant", vol = "GARCH", p = 1, q = 1)
results_garch_1_1 = model_garch_1_1.fit(update_freq = 5)
results_garch_1_1.summary()

Iteration:      5,   Func. Count:     35,   Neg. LLF: 8364.108387217282
Iteration:     10,   Func. Count:     65,   Neg. LLF: 8357.41125648787
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8357.41125648787
            Iterations: 11
            Function evaluations: 69
            Gradient evaluations: 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8357.41
Distribution:                  Normal   AIC:                           16722.8
Method:            Maximum Likelihood   BIC:                           16749.8
                                        No. Observations:                 6199
Date:                Tue, Oct 10 2023   Df Residuals:                     6198
Time:                        00:39:12   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0431  1.048e-02      4.113  3.909e-05 [2.255e-02,6.361e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0132  3.437e-03      3.839  1.233e-04 [6.460e-03,1.993e-02]
alpha[1]       0.0890  1.149e-02      7.746  9.511e-15   [6.645e-02,  0.111]
beta[1]        0.9002  1.297e-02     69.401      0.000     [  0.875,  0.926]
============================================================================

Covariance estimator: robust
"""

## Higher-Lag GARCH Models

In [11]:
model_garch_1_2 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 1, q = 2)
results_garch_1_2 = model_garch_1_2.fit(update_freq = 5)
results_garch_1_2.summary()

Iteration:      5,   Func. Count:     40,   Neg. LLF: 8368.279672293234
Iteration:     10,   Func. Count:     72,   Neg. LLF: 8357.413902628872
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8357.411256569252
            Iterations: 14
            Function evaluations: 95
            Gradient evaluations: 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8357.41
Distribution:                  Normal   AIC:                           16724.8
Method:            Maximum Likelihood   BIC:                           16758.5
                                        No. Observations:                 6199
Date:                Tue, Oct 10 2023   Df Residuals:                     6198
Time:                        00:39:28   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0431  1.047e-02      4.116  3.854e-05 [2.257e-02,6.360e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0132  3.297e-03      4.003  6.265e-05 [6.735e-03,1.966e-02]
alpha[1]       0.0890  1.369e-02      6.501  7.981e-11   [6.214e-02,  0.116]
beta[1]        0.9002      0.206      4.374  1.218e-05     [  0.497,  1.304]
beta[2]    5.5988e-12      0.196  2.851e-11      1.000     [ -0.385,  0.385]
============================================================================

Covariance estimator: robust
"""

In [12]:
model_garch_1_3 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 1, q = 3)
results_garch_1_3 = model_garch_1_3.fit(update_freq = 5)
results_garch_1_3.summary()

Iteration:      5,   Func. Count:     47,   Neg. LLF: 8493.55987174635
Iteration:     10,   Func. Count:     87,   Neg. LLF: 8357.59438793701
Iteration:     15,   Func. Count:    122,   Neg. LLF: 8357.411256447653
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8357.411256447653
            Iterations: 15
            Function evaluations: 122
            Gradient evaluations: 15


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8357.41
Distribution:                  Normal   AIC:                           16726.8
Method:            Maximum Likelihood   BIC:                           16767.2
                                        No. Observations:                 6199
Date:                Tue, Oct 10 2023   Df Residuals:                     6198
Time:                        00:39:33   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0431  2.168e-02      1.987  4.688e-02 [5.948e-04,8.557e-02]
                            Volatility Model                            
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          0.0132  6.831e-02      0.193      0.847 [ -0.121,  0.147]
alpha[1]       0.0890      0.475      0.187      0.851 [ -0.842,  1.020]
beta[1]        0.9002     13.988  6.435e-02      0.949 [-26.516, 28.317]
beta[2]        0.0000     21.329      0.000      1.000 [-41.804, 41.804]
beta[3]    6.0414e-10      7.869  7.677e-11      1.000 [-15.424, 15.424]
========================================================================

Covariance estimator: robust
"""

In [13]:
model_garch_2_1 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 2, q = 1)
results_garch_2_1 = model_garch_2_1.fit(update_freq = 5)
results_garch_2_1.summary()

Iteration:      5,   Func. Count:     41,   Neg. LLF: 8374.273427353817
Iteration:     10,   Func. Count:     76,   Neg. LLF: 8358.743799763204
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8356.284343630054
            Iterations: 14
            Function evaluations: 100
            Gradient evaluations: 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8356.28
Distribution:                  Normal   AIC:                           16722.6
Method:            Maximum Likelihood   BIC:                           16756.2
                                        No. Observations:                 6199
Date:                Tue, Oct 10 2023   Df Residuals:                     6198
Time:                        00:39:37   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0428  1.051e-02      4.075  4.605e-05 [2.224e-02,6.345e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0150  4.700e-03      3.197  1.387e-03  [5.816e-03,2.424e-02]
alpha[1]       0.0710  1.720e-02      4.129  3.645e-05    [3.732e-02,  0.105]
alpha[2]       0.0259  2.449e-02      1.057      0.290 [-2.210e-02,7.389e-02]
beta[1]        0.8908  1.928e-02     46.212      0.000      [  0.853,  0.929]
=============================================================================

Covariance estimator: robust
"""

In [14]:
model_garch_3_1 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 3, q = 1)
results_garch_3_1 = model_garch_3_1.fit(update_freq = 5)
results_garch_3_1.summary()

Iteration:      5,   Func. Count:     45,   Neg. LLF: 15716.336346132946
Iteration:     10,   Func. Count:     85,   Neg. LLF: 8356.31498363358
Iteration:     15,   Func. Count:    119,   Neg. LLF: 8356.284343579697
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8356.284343577969
            Iterations: 15
            Function evaluations: 119
            Gradient evaluations: 15


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8356.28
Distribution:                  Normal   AIC:                           16724.6
Method:            Maximum Likelihood   BIC:                           16765.0
                                        No. Observations:                 6199
Date:                Tue, Oct 10 2023   Df Residuals:                     6198
Time:                        00:39:40   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0428  1.055e-02      4.060  4.909e-05 [2.216e-02,6.353e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0150  5.936e-03      2.532  1.135e-02  [3.394e-03,2.666e-02]
alpha[1]       0.0710  1.720e-02      4.129  3.636e-05    [3.732e-02,  0.105]
alpha[2]       0.0259  2.463e-02      1.051      0.293 [-2.238e-02,7.415e-02]
alpha[3]       0.0000  2.632e-02      0.000      1.000 [-5.159e-02,5.159e-02]
beta[1]        0.8908  2.687e-02     33.145 6.738e-241      [  0.838,  0.943]
=============================================================================

Covariance estimator: robust
"""